### The repository is processing rate map from papers to fit requirement of stdpopsim 

The exact format refer to [msprime.RateMap.read_hapmap](https://tskit.dev/msprime/docs/stable/api.html#msprime.RateMap.read_hapmap) 

#### Johnsson_2021

Johnsson_2021 use the data from the supplyment of [paper](https://doi.org/10.1186/s12711-021-00643-0) 

In additional file has sex-averaged (Additional file 4: Table S3),male-specific (Additional file 2: Table S1), 

and female-specific (Additional file 3: Table S2) rate map. Here, we use the value of sex-averaged rate map.

In [59]:
import msprime
import pandas as pd
import numpy as np

In [60]:
df = pd.read_csv("./Johnsson_2021/Sex-averaged map of the landscape of pig recombination rate in 1-Mb windows.csv", sep=",", header=0)

In [24]:
df

,chr,window_start,window_end,recombination_rate
0,1,1,1000000,0.009154
1,1,1000001,2000000,0.009496
2,1,2000001,3000000,0.006806
3,1,3000001,4000000,0.011100
4,1,4000001,5000000,0.015028
...,...,...,...,...
2267,18,51000001,52000000,0.018101
2268,18,52000001,53000000,0.010463
2269,18,53000001,54000000,0.008005
2270,18,54000001,55000000,0.010662


In [66]:
# Avergae recombination rate for each chromosome
li_rate_mean = []
li_length=[]
for i in range(1,19):
    chr_data=df.groupby('chr').get_group(i)
    r=list(chr_data["recombination_rate"]*1e-6)
    p=[0]
    for j in chr_data["window_end"]:
        p.append(j)

    rate_map = msprime.RateMap(
    position=p,
    rate=r
    )
    li_length.append(rate_map.position[-1])
    li_rate_mean.append(rate_map.mean_rate)
li_rate_mean

[np.float64(5.335247608983185e-09),
 np.float64(8.6057260999254e-09),
 np.float64(9.751141303348547e-09),
 np.float64(9.79460049114614e-09),
 np.float64(1.1992632296111e-08),
 np.float64(8.786173521918499e-09),
 np.float64(1.0887772285689678e-08),
 np.float64(8.923031414594611e-09),
 np.float64(9.337506174170301e-09),
 np.float64(1.654233205589542e-08),
 np.float64(1.2145120574910172e-08),
 np.float64(1.6905062453467094e-08),
 np.float64(6.162796630328349e-09),
 np.float64(8.721569375548237e-09),
 np.float64(8.114468762954893e-09),
 np.float64(1.1407464686635653e-08),
 np.float64(1.3779949516098884e-08),
 np.float64(1.3679923888648167e-08)]

In [ ]:
# Calculate recombination rate and genetic distance for recombination map
# original file the position starts from 1, but msprime requires it to start from 0
# the position of window should be minus 1
# because the msprime.RateMap expects the positions to be inclusive of the start and exclusive of the end
# the end position of the last window should be added 1
# in summary, nothing changes when inputting rate_map
# quatoted from msprime documentation:
# "the first row has a nonzero genetic map position (last column, cM), 
# implying a nonzero recombination rate before that position, that is assumed 
# to extend to the start of the chromosome (at position 0 bp). However, if the first 
# line has a nonzero bp position (second column) and a zero gen"
# So with the 0 position or not, it is does not matter
# here, remove the first position 0,

for i in range(1,19):
    chr_data=df.groupby('chr').get_group(i)
    r=list(chr_data["recombination_rate"]*1e-6)
    p=[0]
    for j in chr_data["window_end"]:
        p.append(j)
    rate_map = msprime.RateMap(
    position=p,
    rate=r
    )
    # Calculate cumulative genetic distance (cM)
    genetic_positions = []
    position = []
    rate_map_list = []
    for j in range(len(rate_map.position)):
        if j == 0:
            position.append(rate_map.position[j])
            genetic_positions.append(0.0)
        elif j == len(rate_map.position) - 1:
            rate_map_list.append(rate_map.rate[j-1])
        else:
            rate_map_list.append(rate_map.rate[j-1])
            position.append(rate_map.position[j])

            # Δp: distance between adjacent physical positions
            delta_p = rate_map.position[j+1] - rate_map.position[j]
            # Δg: genetic distance (in cM)
            delta_g = delta_p * rate_map.rate[j-1] * 100
            genetic_positions.append(genetic_positions[-1] + delta_g)

    # Create DataFrame for genetic map
    # excluding the first position 0
    genetic_map_df=pd.DataFrame({
    "Chromosome": f'chr{i}',
    "Position(bp)": position[1:],
    "Rate(cM/Mb)": np.array(rate_map_list[1:]) * 1e8,  # Convert from M/bp to cM/Mb
    "Map(cM)": genetic_positions[1:]
    })
    genetic_map_df["Position(bp)"]= genetic_map_df["Position(bp)"].astype(int)
    genetic_map_df.to_csv(f"./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr{i}.txt",sep='\t', index=False)

In [48]:
!tar -cvf ./Johnsson_2021/Johnsson_2021_sex_average_rec_map.tar ./Johnsson_2021/Johnsson_2021_sex_average_rec_map
!gzip ./Johnsson_2021/Johnsson_2021_sex_average_rec_map.tar


11228.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr13.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr7.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr6.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr12.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr10.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr4.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr5.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr11.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr1.txt
a ./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr15.txt
a ./Johnsson_2021/Johns

11233.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [72]:
msprime.RateMap.read_hapmap("./Johnsson_2021/Johnsson_2021_sex_average_rec_map/Johnsson_2021_Sex-averaged_map_chr1.txt")

left,right,mid,span,rate
0,1000000,500000,1000000,9.2e-09
1000000,2000000,1500000,1000000,9.5e-09
2000000,3000000,2500000,1000000,6.8e-09
3000000,4000000,3500000,1000000,1.1e-08
4000000,5000000,4500000,1000000,1.5e-08
5000000,6000000,5500000,1000000,3e-08
6000000,7000000,6500000,1000000,2.2e-08
7000000,8000000,7500000,1000000,1.5e-08
8000000,9000000,8500000,1000000,2e-08
9000000,10000000,9500000,1000000,1.4e-08


#### Brekke_2022

In [74]:
df = pd.read_csv("./Brekke_2022/Linkagemap_SusScrofa_Synthetic.txt", sep=" ", header=0)

In [84]:
for i in range(1,19):
    chr_df=df.groupby('chr').get_group(i)
    position = chr_df["bp"].tolist()
    Map = chr_df["cM"].tolist()
    rate_map_list = []
    physical_position = []  # Start with position 0
    genetic_positions = []  # Start with genetic position 0
    for j in range(len(Map)):
        if j < len(Map) - 1:
            physical_position.append(position[j])
            genetic_positions.append(Map[j])
            # Calculate the recombination rate in cM/Mb
            delta_p = position[j+1] - position[j]
            delta_g = Map[j+1] - Map[j]
            rate = delta_g / delta_p
            rate_map_list.append(rate)
        else:
            # last the rate would be missing, to avoid NaN,
            # we use the last position at rate=0
            physical_position.append(position[j])
            genetic_positions.append(Map[j])
            rate_map_list.append(0.0)

    # Create DataFrame for genetic map
    genetic_map_df=pd.DataFrame({
    "Chromosome": f'chr{i}',
    "Position(bp)": physical_position,
    "Rate(cM/Mb)": np.array(rate_map_list) * 1e6,  # Convert from cM/bp to cM/Mb
    "Map(cM)": genetic_positions
    })
    genetic_map_df["Position(bp)"]= genetic_map_df["Position(bp)"].astype(int)
    genetic_map_df.to_csv(f"./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr{i}.txt",sep='\t', index=False)

In [87]:
chr1 = msprime.RateMap.read_hapmap("./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr1.txt")
chr1.mean_rate

np.float64(5.364888573902515e-09)

In [88]:
!tar -cvf ./Brekke_2022/Brekke_2022_synthetic_rec_map.tar ./Brekke_2022/Brekke_2022_synthetic_rec_map
!gzip ./Brekke_2022/Brekke_2022_synthetic_rec_map.tar

16821.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


a ./Brekke_2022/Brekke_2022_synthetic_rec_map
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr18.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr2.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr3.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr1.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr4.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr5.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr7.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr6.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr8.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr9.txt
a ./Brekke_2022/Brekke_2022_synthetic_rec_map/Brekke_2022_synthetic_rec_map_chr14.txt
a ./Brekke_2022/B

16826.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
